In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler



In [3]:
file = 'data/Hz31.txt'
data = pd.read_csv(file, names=['z', 'hz', 'err'], sep = " " )
# besthzorder = np.array([25,  3,  9,  7, 19,  2, 17, 16,  6, 28, 12, 20, 21, 26, 27, 11,  1,
#        10, 30, 13, 29,  0,  8, 18, 24,  5, 23,  4, 22, 15, 14])
# data = data.values[a]

# file = 'data/fs8Diagram.txt'
# data = pd.read_csv(file, names=['z', '$f\sigma_8$', 'err', '$\Omega_ref$'], sep = "\t", skiprows=3)
# data = data[['z', '$f\sigma_8$', 'err']]
# randomize = np.random.permutation(len(data.values))
# a = np.array([21,1,3,10,12,17,8,4,7,16, 2,11,19,13,18,5,9,20,15,14,6,0])
even = [x for x in range(len(data.values)) if x%2 ==0]
odd = [x for x in range(len(data.values)) if x%2 ==1]
even.extend(odd)
order = np.array(even)

data = data.values[order]


X = data[:,0]
y = data[:,1:]

scalerx = StandardScaler()
# scalerz = MinMaxScaler(feature_range=(-1,1))
# fit scaler on data
scalerx.fit(X.reshape(-1,1))
# apply transform
X = scalerx.transform(X.reshape(-1,1))


# scalery = MinMaxScaler(feature_range=(0,1))
# scalery = StandardScaler()
# scalery.fit(y)
# y = scalery.transform(y)

split = 0.8
ntrain = int(split * len(X))
indx = [ntrain]
X_train, X_test = np.split(X, indx)
y_train, y_test = np.split(y, indx)


In [4]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0.01,
                                   patience=50,
                                   restore_best_weights=True)
                                   ]

n_cols = 1

In [5]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([4, 8, 16, 32, 64]))

In [6]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [7]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [8]:
def hzmodel(input_z, hparams):
    #encoder
    efirst = Dense(hparams[HP_NUM_UNITS1], activation='relu', input_shape=(1,))(input_z)
    ehidden = Dense(hparams[HP_NUM_UNITS2], activation='relu')(efirst)
#     ehidden2 = Dense(hparams[HP_NUM_UNITS3], activation='relu')(ehidden1)
    elast = Dense(2, activation='relu')(ehidden)
        
    return elast

In [9]:
epochs = 200
input_z = tf.keras.layers.Input(shape = (1,))

In [10]:
def train_test_model(hparams):
    model = Model(input_z, hzmodel(input_z, hparams))  
                              
    model.compile(
#             optimizer=hparams[HP_OPTIMIZER],
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(X_train, y_train, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks, verbose=0) 
    _, loss = model.evaluate(X_test, y_test)


    return loss
    

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
#     with tf.summary.FileWriter(run_dir):
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [13]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
#             for num_units3 in HP_NUM_UNITS3.domain.values:
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#                     for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
              HP_NUM_UNITS1: num_units1,
              HP_NUM_UNITS2: num_units2,
#               HP_NUM_UNITS3: num_units3,
            #                       HP_DROPOUT: dropout_rate,
#                       HP_OPTIMIZER: optimizer,
              HP_BATCHSIZE: batch_size
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
#                 print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

print("Total combinations: {}".format(session_num+1))

--- Starting trial: run-0
1/1 [==============================] - 0s 1ms/step - loss: 10711.0049 - mean_squared_error: 10711.0049
--- Starting trial: run-1
1/1 [==============================] - 0s 1ms/step - loss: 351.7194 - mean_squared_error: 351.7194
--- Starting trial: run-2
1/1 [==============================] - 0s 785us/step - loss: 353.6345 - mean_squared_error: 353.6345
--- Starting trial: run-3
1/1 [==============================] - 0s 1ms/step - loss: 676.6929 - mean_squared_error: 676.6929
--- Starting trial: run-4
1/1 [==============================] - 0s 844us/step - loss: 676.6689 - mean_squared_error: 676.6689
--- Starting trial: run-5
1/1 [==============================] - 0s 799us/step - loss: 359.6026 - mean_squared_error: 359.6026
--- Starting trial: run-6
1/1 [==============================] - 0s 741us/step - loss: 10711.9512 - mean_squared_error: 10711.9512
--- Starting trial: run-7
1/1 [==============================] - 0s 956us/step - loss: 365.2331 - mean_square

1/1 [==============================] - 0s 837us/step - loss: 361.4492 - mean_squared_error: 361.4492
--- Starting trial: run-13
1/1 [==============================] - 0s 1ms/step - loss: 10711.5537 - mean_squared_error: 10711.5537
--- Starting trial: run-14
1/1 [==============================] - 0s 725us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-15
1/1 [==============================] - 0s 1ms/step - loss: 348.6262 - mean_squared_error: 348.6262
--- Starting trial: run-16
1/1 [==============================] - 0s 881us/step - loss: 375.7964 - mean_squared_error: 375.7964
--- Starting trial: run-17
1/1 [==============================] - 0s 752us/step - loss: 375.6009 - mean_squared_error: 375.6009
--- Starting trial: run-18
1/1 [==============================] - 0s 1ms/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-19
1/1 [==============================] - 0s 738us/step - loss: 367.8233 - mean_squared_error: 367.82

1/1 [==============================] - 0s 1ms/step - loss: 360.9931 - mean_squared_error: 360.9931
--- Starting trial: run-23
1/1 [==============================] - 0s 943us/step - loss: 352.6935 - mean_squared_error: 352.6935
--- Starting trial: run-24
1/1 [==============================] - 0s 838us/step - loss: 368.4552 - mean_squared_error: 368.4552
--- Starting trial: run-25
1/1 [==============================] - 0s 823us/step - loss: 683.5995 - mean_squared_error: 683.5995
--- Starting trial: run-26
1/1 [==============================] - 0s 999us/step - loss: 360.2841 - mean_squared_error: 360.2841
--- Starting trial: run-27
1/1 [==============================] - 0s 907us/step - loss: 360.4627 - mean_squared_error: 360.4627
--- Starting trial: run-28
1/1 [==============================] - 0s 679us/step - loss: 364.7044 - mean_squared_error: 364.7044
--- Starting trial: run-29
1/1 [==============================] - 0s 723us/step - loss: 358.8504 - mean_squared_error: 358.8504
--- S

1/1 [==============================] - 0s 748us/step - loss: 454.0043 - mean_squared_error: 454.0043
--- Starting trial: run-33
1/1 [==============================] - 0s 1ms/step - loss: 724.6381 - mean_squared_error: 724.6381
--- Starting trial: run-34
1/1 [==============================] - 0s 809us/step - loss: 695.9373 - mean_squared_error: 695.9373
--- Starting trial: run-35
1/1 [==============================] - 0s 965us/step - loss: 385.1003 - mean_squared_error: 385.1003
--- Starting trial: run-36
1/1 [==============================] - 0s 775us/step - loss: 390.1357 - mean_squared_error: 390.1357
--- Starting trial: run-37
1/1 [==============================] - 0s 757us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-38
1/1 [==============================] - 0s 829us/step - loss: 346.2776 - mean_squared_error: 346.2776
--- Starting trial: run-39
1/1 [==============================] - 0s 859us/step - loss: 351.0617 - mean_squared_error: 351.0617
-

1/1 [==============================] - 0s 808us/step - loss: 351.6002 - mean_squared_error: 351.6002
--- Starting trial: run-43
1/1 [==============================] - 0s 800us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-44
1/1 [==============================] - 0s 724us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-45
1/1 [==============================] - 0s 1ms/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-46
1/1 [==============================] - 0s 929us/step - loss: 362.6123 - mean_squared_error: 362.6123
--- Starting trial: run-47
1/1 [==============================] - 0s 774us/step - loss: 362.4458 - mean_squared_error: 362.4458
--- Starting trial: run-48
1/1 [==============================] - 0s 1ms/step - loss: 4246.4556 - mean_squared_error: 4246.4556
--- Starting trial: run-49
1/1 [==============================] - 0s 709us/step - loss: 10716.1016 - mean_squared_error: 

1/1 [==============================] - 0s 1ms/step - loss: 2048.8342 - mean_squared_error: 2048.8342
--- Starting trial: run-53
1/1 [==============================] - 0s 885us/step - loss: 509.2627 - mean_squared_error: 509.2627
--- Starting trial: run-54
1/1 [==============================] - 0s 969us/step - loss: 472.0565 - mean_squared_error: 472.0565
--- Starting trial: run-55
1/1 [==============================] - 0s 703us/step - loss: 463.1229 - mean_squared_error: 463.1229
--- Starting trial: run-56
1/1 [==============================] - 0s 756us/step - loss: 602.5401 - mean_squared_error: 602.5401
--- Starting trial: run-57
1/1 [==============================] - 0s 772us/step - loss: 440.4107 - mean_squared_error: 440.4107
--- Starting trial: run-58
1/1 [==============================] - 0s 1ms/step - loss: 766.1579 - mean_squared_error: 766.1579
--- Starting trial: run-59
1/1 [==============================] - 0s 729us/step - loss: 417.2417 - mean_squared_error: 417.2417
--- S

1/1 [==============================] - 0s 899us/step - loss: 10712.0732 - mean_squared_error: 10712.0732
--- Starting trial: run-63
1/1 [==============================] - 0s 837us/step - loss: 423.4973 - mean_squared_error: 423.4973
--- Starting trial: run-64
1/1 [==============================] - 0s 791us/step - loss: 10717.5186 - mean_squared_error: 10717.5186
--- Starting trial: run-65
1/1 [==============================] - 0s 940us/step - loss: 11034.5000 - mean_squared_error: 11034.5000
--- Starting trial: run-66
1/1 [==============================] - 0s 1ms/step - loss: 457.1973 - mean_squared_error: 457.1973
--- Starting trial: run-67
1/1 [==============================] - 0s 810us/step - loss: 535.4213 - mean_squared_error: 535.4213
--- Starting trial: run-68
1/1 [==============================] - 0s 2ms/step - loss: 1536.2209 - mean_squared_error: 1536.2209
--- Starting trial: run-69
1/1 [==============================] - 0s 708us/step - loss: 612.9308 - mean_squared_error: 61

1/1 [==============================] - 0s 944us/step - loss: 1629.5410 - mean_squared_error: 1629.5410
--- Starting trial: run-73
1/1 [==============================] - 0s 1ms/step - loss: 433.2873 - mean_squared_error: 433.2873
--- Starting trial: run-74
1/1 [==============================] - 0s 793us/step - loss: 436.8315 - mean_squared_error: 436.8315
--- Starting trial: run-75
1/1 [==============================] - 0s 2ms/step - loss: 751.9692 - mean_squared_error: 751.9692
--- Starting trial: run-76
1/1 [==============================] - 0s 1ms/step - loss: 625.9243 - mean_squared_error: 625.9243
--- Starting trial: run-77
1/1 [==============================] - 0s 864us/step - loss: 436.9706 - mean_squared_error: 436.9706
--- Starting trial: run-78
1/1 [==============================] - 0s 992us/step - loss: 422.9897 - mean_squared_error: 422.9897
--- Starting trial: run-79
1/1 [==============================] - 0s 732us/step - loss: 401.4480 - mean_squared_error: 401.4480
Total c

In [15]:
!kill 7439
%tensorboard --logdir logs/hparam_tuning